# Make Azure Blobs Publicly Accessible

## Preparation

In [7]:
#r "nuget: Azure.Storage.Blobs"

Installed Packages Azure.Storage.Blobs, 12.25.0

Import namespaces:

In [8]:
using System.IO;
using Azure.Storage.Blobs;
using Azure.Storage.Blobs.Models;

Set up connection string:

In [ ]:
string connectionString = "DefaultEndpointsProtocol=https;AccountName=ztechtest;AccountKey=XXX;EndpointSuffix=core.windows.net";

## Usage

Create container with public access:

In [10]:
// Create a BlobServiceClient using the connection string
var blobServiceClient = new BlobServiceClient(connectionString);

// Create new container name
var containerName = "public";

// Check if the container exists
var containerClient = blobServiceClient.GetBlobContainerClient(containerName);
if (await containerClient.ExistsAsync()) {
    // Set public access if it already exists
    await containerClient.SetAccessPolicyAsync(PublicAccessType.Blob);
    Console.WriteLine($"Container '{containerName}' setup for public anonymous access.");
} else {
    // Create the container with public access
    containerClient = await blobServiceClient.CreateBlobContainerAsync(
        containerName,
        PublicAccessType.Blob
    );
    Console.WriteLine($"Container '{containerName}' created with public anonymous access.");
}

Container 'public' created with public anonymous access.


* `PublicAccessType.BlobContainer` allows anonymous/public read access to both the blobs and the container metadata (like listing blobs in the container).
* `PublicAccessType.Blob` allows anonymous/public read access only to the blobs themselves. The container metadata (such as listing blobs) is not accessible publicly.

Upload some pictures:

In [11]:
var directoryPath = @"C:\Windows\Web\Wallpaper\ThemeA";
var jpgFiles = Directory.GetFiles(directoryPath, "*.jpg");
foreach (var filePath in jpgFiles) {
    var fileName = Path.GetFileName(filePath);
    var fileBlobClient = containerClient.GetBlobClient(fileName);

    using (var fileStream = File.OpenRead(filePath)) {
        // Add headers to allow direct display in browser and cache for a year
        var blobHttpHeaders = new BlobHttpHeaders { 
            ContentType = "image/jpeg",
            CacheControl = "public, max-age=31536000"
        };
        // Upload the file with the specified headers
        await fileBlobClient.UploadAsync(fileStream, blobHttpHeaders);

        // Output the URL of the uploaded file
        Console.WriteLine(fileBlobClient.Uri);
    }
}

https://ztechtest.blob.core.windows.net/public/img20.jpg
https://ztechtest.blob.core.windows.net/public/img21.jpg
https://ztechtest.blob.core.windows.net/public/img22.jpg
https://ztechtest.blob.core.windows.net/public/img23.jpg


The URLs generated above are publicly accessible, anyone who knows them can download the image.

## Cleanup

Delete the container and all its blobs:

In [12]:
if (await containerClient.ExistsAsync()) {
    await containerClient.DeleteAsync();
    Console.WriteLine($"Container '{containerName}' and all its blobs have been deleted.");
} else {
    Console.WriteLine($"Container '{containerName}' does not exist.");
}

Container 'public' and all its blobs have been deleted.
